## **Step 1: Installing dependencies**

In [ ]:
!pip install torch transformers peft datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 13.5 MB/s eta 0:00:00


**CUDA MEMORY RESET**

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

# **Step 2: Logging into hugging face**

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `kishanmodel` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-c

# **Step 3: Finetuning with QLoRA**

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
import os
import json

# --- Configuration ---
model_name = "google/gemma-3-4b-it"  # 4B params model
dataset_path = "cvpr_finetuning_formatted.jsonl"
new_model_name = "gemma-3-4b-cvpr"  # Adapter output
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# QLoRA config
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# BitsAndBytes (4-bit)
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Training args (for Colab T4 ~16GB VRAM)
output_dir = "./results"
num_train_epochs = 1
fp16 = True
bf16 = False
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_8bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
dataset = load_dataset('json', data_files=dataset_path, split="train")

# --- Quantization Config ---
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# --- Load Model ---
print(f"Loading {model_name}...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# --- LoRA Config ---
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules
)

# --- Training Args ---
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
)

# --- SFTTrainer ---
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
)

# --- Train ---
print("Starting fine-tuning...")
trainer.train()

# --- Save Adapter ---
print(f"Saving adapter to {new_model_name}...")
trainer.model.save_pretrained(new_model_name)

print("Fine-tuning complete! Adapter saved.")

# **Step 4: Merge the fine tuned parameters with Base Model**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_name = "google/gemma-3-4b-it"
adapter_path = "gemma-3-4b-cvpr"
merged_model_path = "/content/drive/MyDrive/gemma-3-4b-cvpr-merged"

# Load base
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Load + merge PEFT
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()

# Save merged
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("Merging done saving to google drive")
# Save to Google Drive
!cp -r {merged_model_path} /content/drive/MyDrive/

print("Merged model saved!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Merging done saving to google drive
cp: '/content/drive/MyDrive/gemma-3-4b-cvpr-merged' and '/content/drive/MyDrive/gemma-3-4b-cvpr-merged' are the same file
Merged model saved!


In [ ]:
#zipping and downloading the folder to save progress
from google.colab import files

# The name of the folder you want to download
folder_name = 'gemma-3-4b-cvpr'

# The name of the zip file that will be created
zip_file_name = f'{folder_name}.zip'

# 1. Zip the folder
# The '-r' flag means it will include all subdirectories and files
!zip -r {zip_file_name} {folder_name}

# 2. Download the created zip file
files.download(zip_file_name)

print(f"Successfully zipped '{folder_name}' and initiated the download.")

# **Step 5: Converting the model into gguf format**

In [ ]:
!apt-get update

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,937 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,544 kB

In [ ]:
#installing build essentials
!apt-get install -y build-essential cmake git g++ make

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 59894, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 59894 (delta 111), reused 60 (delta 59), pack-reused 59741 (from 4)
Receiving objects: 100% (59894/59894), 157.36 MiB | 27.08 MiB/s, done.
Resolving deltas: 100% (43079/43079), done.


In [ ]:
#building build file
!cd llama.cpp && mkdir -p build && cd build && cmake .. -G Ninja -DLLAMA_BUILD_SERVER=ON -DLLAMA_CUDA=OFF && ninja

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found versio

In [ ]:
!pip install -r llama.cpp/requirements/requirements-convert_hf_to_gguf.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment


In [ ]:
#Converting the model into gguf fomat along with dequantization to 16 bit
!python llama.cpp/convert_hf_to_gguf.py /content/drive/MyDrive/gemma-3-4b-cvpr-merged --outfile /content/gemma-3-4b-cvpr-f16.gguf --outtype f16

INFO:hf-to-gguf:Loading model: gemma-3-4b-cvpr-merged
INFO:hf-to-gguf:Model architecture: Gemma3ForConditionalGeneration
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:token_embd.weight,                 torch.float16 --> F16, shape = {2560, 262208}
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.float16 --> F32, shape = {2560}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.float16 --> F16, shape = {10240, 2560}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,             torch.float16 --> F16, shape = {2560, 10240}
INFO:hf-to-gguf:blk.0.ffn_up.weight,               torch.float16 --> F16, shape = {2560, 10240}
INFO:hf-to-gguf:blk.0.post_attention_norm.weight,  torch.float16 --> F32, shape = {2560}
INFO:hf-to-gguf:blk.0.post_ffw_norm.weight,        to

In [ ]:
!ls -lh llama.cpp/build/bin/llama-quantize

-rwxr-xr-x 1 root root 355K Aug 18 19:49 llama.cpp/build/bin/llama-quantize


In [ ]:
#Converting the 16 bit gguf formatted model to 4 bit gguf formatted model
!llama.cpp/build/bin/llama-quantize /content/drive/MyDrive/gemma-3-4b-cvpr-f16.gguf /content/drive/MyDrive/gemma-3-4b-cvpr-q4_k_m.gguf q4_k_m

main: build = 6198 (6d7f1117)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/drive/MyDrive/gemma-3-4b-cvpr-f16.gguf' to '/content/drive/MyDrive/gemma-3-4b-cvpr-q4_k_m.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 35 key-value pairs and 444 tensors from /content/drive/MyDrive/gemma-3-4b-cvpr-f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gemma 3 4b Cvpr Merged
llama_model_loader: - kv   3:                           general.finetune str              = cvpr-merged
llama_model_loader: - kv   4:                           general.basename str           